In [1]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_test=pd.read_csv("spaceship_test.csv")

檢查缺失值

In [5]:
df_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

使用訓練模型時相同的方式對test data進行資料前處理

In [6]:
list_missing = list((df_test.select_dtypes(['object']).isna().sum() > 0).index)
for col in list_missing:
    df_test[col] = df_test[col].fillna(df_test[col].mode()[0])

In [7]:
df_test[["CabinDeck", "CabinNo.", "CabinSide"]] = df_test["Cabin"].str.split('/', expand = True)
df_test[["Group", "NuminGroup"]] = df_test["PassengerId"].str.split('_', expand = True)
df_test.drop("NuminGroup",axis=1,inplace=True)
df_test.drop('Cabin',axis=1,inplace=True)
df_test.drop('PassengerId',axis=1,inplace=True)

In [8]:
df_test.drop('Name',axis=1,inplace=True)

In [9]:
HomePlanet_mapping={"Earth":0,"Europa":1,"Mars":2}
df_test['HomePlanet'] = df_test['HomePlanet'].map(HomePlanet_mapping)
Destination_mapping={"TRAPPIST-1e":0,"55 Cancri e":1,"PSO J318.5-22":2}
df_test['Destination'] = df_test['Destination'].map(Destination_mapping)
CabinDeck_mapping={"A":0,"B":1,"C":2,"D":3,"E":4,"F":5,"G":6,"T":7}
df_test['CabinDeck'] = df_test['CabinDeck'].map(CabinDeck_mapping)


In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_test['CabinNo.']=le.fit_transform(df_test['CabinNo.'])
CabinSide_mapping={"S":0,"P":1}
df_test['CabinSide'] = df_test['CabinSide'].map(CabinSide_mapping)
df_test['Group']=le.fit_transform(df_test['Group'])

In [14]:
df_test['Age'].fillna(df_test['Age'].median(),inplace=True)
df_test['RoomService'].fillna(0,inplace=True)
df_test['FoodCourt'].fillna(0,inplace=True)
df_test['ShoppingMall'].fillna(0,inplace=True)
df_test['Spa'].fillna(0,inplace=True)
df_test['VRDeck'].fillna(0,inplace=True)

確認補值完畢

In [15]:
df_test.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
CabinDeck       0
CabinNo.        0
CabinSide       0
Group           0
dtype: int64

載入訓練好的模型

In [20]:
import joblib
model_pretrained=joblib.load('Spaceship-LR.pk1')
predictions=model_pretrained.predict(df_test)
predictions

array([ True, False,  True, ...,  True,  True,  True])

In [29]:
df_sub=pd.read_csv("spaceship_test.csv")

依照繳交格式新增新的dataframe

In [30]:
forSubmissionDF = pd.DataFrame(columns=['PassengerId','Transported'])
forSubmissionDF['PassengerId']=df_sub['PassengerId']


將預測結果加入新的dataset

In [31]:
forSubmissionDF['Transported']=predictions
forSubmissionDF

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [33]:
forSubmissionDF.to_csv('for_submission.csv',index=False)

# Kaggle競賽分數

Public score : 0.78

# 改善方向

若要提高競賽分數及模型準確度，未來可以有以下幾種進步方向 : 
1. 資料前處理補值方法 : 可以嘗試其他方式進行補值，例如:補0改為以中位數補值，先將資料集分組後再以各組的中間值進行補值，像是課程中的titanic預測中使用性別分組，但因為此資料集並沒有較直覺的分組方式，所以我覺得思考如何將其分組也是未來可已進步改善的地方。或是若缺失值佔資料集總比例不多，可以直接將有缺失值的那筆資料刪除，上課時有提到，因為補值算是虛假的資料，可能無法貼合資料真實的情況，或多或少都會影響到模型預測的效果，所以有時直接將有缺失值的資料刪除模型準確度會較高。
2. 更改模型 : 本次競賽使用的模型為LogisticRegression，可以多用幾種模型實驗，選出分數最高的模型。
例如:Decision Tree、Random Forest或是Kaggle比賽中很多人使用的XGBoost，它的優點是有很多的參數可以調整，可以找到最符合資料集的參數設定。
3. 進行欄位篩選 : 目前使用了14個欄位進行模型訓練，可以對資料及進行特徵篩選，使用RFECV等方法，選出分數最高，訓練模型效果最好的特徵子集合做為模型訓練使用，不需要用到所有的欄位。或是可以對欄位做處理，像是RoomService/FoodCourt/ShoppingMall/Spa/VRDeck 5個欄位是乘客在Titanic上的消費金額，可以想辦法將其合併或是作金額區段編碼。
